In [1]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [2]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [3]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

### Cleaning Up Sentence

In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

### Sentence to Bag of words

In [5]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

### Predict 

In [6]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

### Chat Bot Response Code

In [7]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
print("Aleeza Bot! is running")
while(True):
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

Aleeza Bot! is running
hey
1/1 [==============================] - 0s 113ms/step
How may I assist you today?
i'm feeling a bit down
1/1 [==============================] - 0s 33ms/step
Be positive! Everything will be fine!
what's your name?
1/1 [==============================] - 0s 29ms/step
My name is Aleeza the Chatbot! and I'm a chat bot!
who made you?
1/1 [==============================] - 0s 28ms/step
My owner is Faiq Ahmad
what's your age?
1/1 [==============================] - 0s 29ms/step
I'm 22 years old.
i love u
1/1 [==============================] - 0s 29ms/step
As an AI chatbot I can't have any feelings for you I'm sorry!
how old are you?
1/1 [==============================] - 0s 29ms/step
I'm 22 years old.
your sleep cycle?
1/1 [==============================] - 0s 30ms/step
As an AI chatbot I don't sleep
I have to go now bye
1/1 [==============================] - 0s 27ms/step
It was great talking to you! Bye
